# Import libraries
Import libraries used in notebook and the mike hydro data handler. 
Make sure the "mHydro_data_handler.py" file is located in the same folder as this notebook.

In [1]:
import pandas as pd
from mHydro_data_handlers import routing_methods
data_handler=routing_methods.MHydroDataHandler()

## load model setup
Load the mike hydro model. After updating the model it can be saved to a new mike hydro model file. 

In [2]:
template_filename=".\model_example\WaterQuality_HD_only.mhydro"

In [3]:
data_handler.load_data(template_filename)

# create update file from setup
This can be used to generate a parameter update file with the right columns and all the nodes in the configuration.
Options that can be used for updating follow the naming in mike hydro.

water level methods:
- manning
- ratingcurve (must be configured afterwards in mike hydro gui)
- none

Flow Routing method:
- translation
- linearreservoir
- muskingum
- norouting

The "Branch" is only included in the update file as a help but only the River node identifier is used for updating. 

In [6]:
export_template = False
if export_template:
    data_handler.write_update_template(r".\model_example\routhing_methods_update.csv")

# read update data
Load the csv file used to update the model. The name of the columns must follow the id's used in the function update_wq_parameters used below.

In [8]:
update_csv_filename=r".\model_example\routhing_methods_update.csv"
update_data=pd.read_csv(update_csv_filename, index_col=False, keep_default_na=False)
update_data.columns = update_data.columns.str.strip()
display(update_data.head(5))

,Branch,River node,Chainage,ID,Flow routing method,Delay,Shape,Water level method,ManningN,Slope,ManningMaxDepth,Width,Type
0,E6,N5,0.00000,Method 1,norouting,2.400000e-09,0.2,manning,0.10,0.05,5,12,CatchmentRoutingMethodPriority
1,E6,N2,59923.96821,Method 4,muskingum,1.000000e+01,1.0,manning,0.10,0.02,5,10,Catchment
2,E5,N4,0.00000,Method 5,muskingum,1.000000e+01,1.0,manning,0.10,0.02,5,10,Regular
3,E3,N1,0.00000,Method 2,norouting,2.400000e-09,0.2,manning,0.05,0.02,5,10,RoutingMethod
4,E2,N3,0.00000,Method 3,norouting,2.400000e-09,0.2,manning,0.05,0.02,5,10,RoutingMethod


### example of modifying update data in notebook

In [ ]:
if False:
    row_selection=update_data["Flow routing method"]=="translation"
    column_selection=["Flow routing method"]
    update_data.loc[row_selection, column_selection]="muskingum"
    display(update_data)

## update model and save to new model file

In [9]:
for index, row in update_data.iterrows():
    par_dict= data_handler.get_parameter_dictionary(nodeId=row['River node'], description=row['ID'], routing_type=row['Flow routing method'], 
                                                    delay=row['Delay'], shape=row['Shape'], water_level_method=row['Water level method'], 
                                                    manning_N=row['ManningN'], width=row['Width'], slope=row['Slope'], 
                                                    manning_max_depth=row['ManningMaxDepth'])
    data_handler.update_node(par_dict)

### save to file

In [10]:
output_file_name=r".\model_example\WaterQuality_HD_only_routing_method_updated.mhydro"
data_handler.write_to_file(output_file_name)

# Utilities

In [4]:
# clear routing methods from setup. Save again afterwards to persist
if False:
    mh_data = data_handler.get_mHydro_data()
    mh_data.KinematicRoutingMethodList.Clear()